In [22]:
import pandas as pd
import os 

player_filename = 'game_summary_by_player.csv'
player_dir = os.getcwd().replace('/notebooks', '') + '/data/external'
player_path = os.path.join(player_dir, player_filename)

player_game_df = pd.read_csv(player_path)
players_df = player_game_df[["player_link", "player_name"]].drop_duplicates()
players_df.head()

/Users/greg.bolla/.local/share/virtualenvs/nfl-stat-scraper-Jt7XuzPt/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,player_link,player_name
0,/players/F/FreeDo20.htm,Doug Free
1,/players/L/LiviNa20.htm,Nate Livings
2,/players/B/BernMa20.htm,Mackenzy Bernadeau
3,/players/S/SmitTy00.htm,Tyron Smith
4,/players/R/RomoTo00.htm,Tony Romo


In [51]:
from bs4 import BeautifulSoup, NavigableString
import requests
import re
import time

def get_html(url):
    wait_time = 1
    html = requests.get(url).content.decode("utf-8") 
    soup = BeautifulSoup(re.sub("<!--|-->","", html), "lxml")  
    time.sleep(wait_time)
    
    return soup

In [91]:
from datetime import datetime
base_url = 'https://www.pro-football-reference.com'

player_url = '/players/L/LiviNa20.htm'
url = "{}{}".format(base_url, player_url)
player_soup = get_html(url)

player_dict = {}
birthdate_string = player_soup.find("span", {"itemprop":"birthDate"})['data-birth']
birthdate = datetime.strptime(birthdate_string, '%Y-%m-%d')
player_dict['birthdate'] = birthdate
print(birthdate)


1982-03-16 00:00:00


In [33]:
combine_tbl = player_soup.find("table", {"id": "combine"})
combine_row = combine_tbl.tbody.tr
print(combine_row)

combine_yr = combine_row.find("th", {"data-stat" : "year_id"}).a.string
combine_height = combine_row.find("td", {"data-stat" : "height"}).string
combine_weight = combine_row.find("td", {"data-stat" : "weight"}).string
combine_forty = combine_row.find("td", {"data-stat" : "forty_yd"}).string
combine_bench = combine_row.find("td", {"data-stat" : "bench_reps"}).string
combine_broad_jump = combine_row.find("td", {"data-stat" : "broad_jump"}).string
combine_shuttle = combine_row.find("td", {"data-stat" : "shuttle"}).string
combine_cone = combine_row.find("td", {"data-stat" : "cone"}).string
combine_vert = combine_row.find("td", {"data-stat" : "vertical"}).string

print(combine_yr, combine_height, combine_weight, combine_forty, combine_bench, \
      combine_broad_jump, combine_shuttle, combine_cone, combine_vert)

<tr><th class="left " data-stat="year_id" scope="row"><a href="/draft/2003-combine.htm">2003</a></th><td class="left " data-stat="pos">QB</td><td class="right " data-stat="height">74</td><td class="right " data-stat="weight">230</td><td class="right " data-stat="forty_yd">5.01</td><td class="right " data-stat="bench_reps">0</td><td class="right " data-stat="broad_jump">105</td><td class="right " data-stat="shuttle">4.20</td><td class="right " data-stat="cone">7.11</td><td class="right " data-stat="vertical">30.0</td></tr>
2003 74 230 5.01 0 105 4.20 7.11 30.0


In [92]:
player_meta = player_soup.find("div", {"itemtype" : "https://schema.org/Person"})

draft_pattern = re.compile(r'Draft')
player_meta.find('strong', text=draft_pattern)
draft_tag = player_meta.find('strong', text=draft_pattern).parent

for child in draft_tag.children:

    # check to see if child is a string, and check to see if there are parentheses within that string
    if isinstance(child, NavigableString) and re.search(r'\((.*?)\)',child) is not None:
        
        re_draft_pick = re.search(r'\((.*?)\)',child)            
        draft_pick_string = re_draft_pick.group(1)

        # in the string that specifies draft pick, grab the number before ordinal string
        draft_pick = re.search(r'.+?(?=[st|nd|rd|th])', draft_pick_string).group(0)
        draft_pick = int(draft_pick)
        print(draft_pick)


AttributeError: 'NoneType' object has no attribute 'parent'

In [90]:
pattern = re.compile(r'Position')
tag = player_meta.find('strong', text=pattern).parent

for child in tag:
    if isinstance(child, NavigableString) and ":" in child:
        position = child.split(":")[-1].strip()

G
